# Homework #3 

Matriculation number: A0153347A
Email: e0025553@nus.edu.sg

Matriculation number: A0097689Y
Email: a0097689@u.nus.edu.sg

Matriculation number: A0153196B
Email: e0025302@nus.edu.sg


### General Notes about this assignment 

Data:
X_train.npy: training data numpy array. Each row in the array corresponds to an image unrolled to a vector (50 x 37 = 1850 dimension)

y_train.npy: labels (0-6) of each data corresponding to the image in the same row in X_train.npy

X_test.npy: testing data numpy array for evaluation of your models. The prediction outputs on this test data is to be submitted for scoreboard

Ideas:
- multiclass SVM (see Multiclass classification)
- kNN with PCA 
- Dense (fully connected neural network)
- CNN (convolutional neural network)

# Files included in this submission

- hw3-lfw.ipynb
- readme.pdf
- X_train.npy
- Y_train.npy
- X_test.npy

## Setting up libraries and packages

Setting up Keras, Theano, Lasagne for Mac OS-X users, assuming you have conda and anaconda set up already.

1. conda update conda
2. Install lasagne: pip install lasagne
3. Install the latest version of Theano, pip install git+git://github.com/Theano/Theano.git
4. Install the latest version of keras, pip install git+git://github.com/fchollet/keras.git

If the above doesn't work, its probably because you have multiple installations of python on your machine, it is installing into the global site-packages of your other python executable. To overcome this, create a conda environment before installing packages into it, run these steps before running the steps above:

1. Create a new conda env - conda create --name hw3
2. source activate hw3


In [67]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import History
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

In [68]:
#Load dataset

#train data

xin= np.load("./X_train.npy")
yin= np.load("./y_train.npy")
xout= np.load("./X_test.npy")
enc = OneHotEncoder()


In [12]:
# Helper functions
def createSubmission(filename, y):
    fo = open( filename , 'w' )
    fo.write("ImageId,PredictedClass\n")
    for i in range(y.shape[0]):
        fo.write(str(i)+","+str(y[i])+"\n")
    fo.close()

In [24]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(1, 50, 37), activation='relu'))
model.add(Convolution2D(32, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(7))
model.add(Activation('softmax'))

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['fbeta_score'])

history = History()
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

# Split the train and test - we want to train on 80% of the data, and 20% for testing
skf = StratifiedKFold(n_splits=5)
for train, test in skf.split(xin, yin):
    train = train
    test = test
    
x_train = np.reshape(xin[train], (-1,1,50,37), order='C')
x_test  = np.reshape(xin[test], (-1,1,50,37), order='C')
y_test  = np.reshape(yin[test], (-1,1))
y_train = np.reshape(yin[train], (-1,1))
enc.fit(y_test)
y_test  = enc.transform(y_test).toarray()
enc.fit(y_train)
y_train = enc.transform(y_train).toarray()
model.fit(x_train, y_train, batch_size=32, nb_epoch=400, validation_data=(x_test, y_test), verbose=1, callbacks=[history, earlystopping])

Train on 774 samples, validate on 192 samples
Epoch 1/400
774/774 [==============================] - 11s - loss: 10.8834 - fbeta_score: 0.2662 - val_loss: 9.4862 - val_fbeta_score: 0.4115
Epoch 2/400
774/774 [==============================] - 10s - loss: 9.4773 - fbeta_score: 0.3902 - val_loss: 9.4862 - val_fbeta_score: 0.4115
Epoch 3/400
774/774 [==============================] - 11s - loss: 9.4723 - fbeta_score: 0.4057 - val_loss: 9.4862 - val_fbeta_score: 0.4115
Epoch 4/400
774/774 [==============================] - 10s - loss: 9.4749 - fbeta_score: 0.3966 - val_loss: 9.4862 - val_fbeta_score: 0.4115
Epoch 5/400
774/774 [==============================] - 11s - loss: 9.4345 - fbeta_score: 0.3685 - val_loss: 9.4862 - val_fbeta_score: 0.4115
Epoch 6/400
774/774 [==============================] - 12s - loss: 9.3365 - fbeta_score: 0.3863 - val_loss: 9.4852 - val_fbeta_score: 0.4115
Epoch 7/400
774/774 [==============================] - 10s - loss: 8.5638 - fbeta_score: 0.3544 - val_loss:

In [63]:
img_xin   = np.reshape(xin, (-1,1,50,37), order='C')
img_xout  = np.reshape(xout, (-1,1,50,37), order='C')
yin   = np.reshape(yin, (-1,1))
enc.fit(yin)
yin   = enc.transform(yin).toarray()
history_all = History()
earlystopping_all = EarlyStopping(monitor='loss', min_delta=0, patience=5)
model.fit(img_xin, yin, nb_epoch=25, batch_size=32, callbacks=[history_all, earlystopping_all])

Epoch 1/25
966/966 [==============================] - 12s - loss: 0.0529 - fbeta_score: 0.9839    
Epoch 2/25
966/966 [==============================] - 12s - loss: 0.0298 - fbeta_score: 0.9880    
Epoch 3/25
966/966 [==============================] - 12s - loss: 0.0499 - fbeta_score: 0.9819    
Epoch 4/25
966/966 [==============================] - 12s - loss: 0.0481 - fbeta_score: 0.9803    
Epoch 5/25
966/966 [==============================] - 11s - loss: 0.0344 - fbeta_score: 0.9891    
Epoch 6/25
966/966 [==============================] - 12s - loss: 0.0428 - fbeta_score: 0.9859    
Epoch 7/25
966/966 [==============================] - 11s - loss: 0.0750 - fbeta_score: 0.9761    
Epoch 8/25
966/966 [==============================] - 12s - loss: 0.0548 - fbeta_score: 0.9865    


In [64]:
img_xout = np.reshape(xout, (322,1,50,37), order='C')
print(img_xout.shape)
y_pred = model.predict(img_xout, batch_size=64, verbose=1)
print(y_pred.shape)


(322, 1, 50, 37)
322/322 [==============================] - 1s     
(322, 7)


In [65]:
# Convert from one-hot encoding to integer labels
y_predicted = np.argmax(y_pred, axis=1)

In [66]:
createSubmission("submission_cnn_11_1_3.csv", y_predicted)

## Statement of Individual Work

Please initial (between the square brackets) one of the following statements.

I, [FP] A0153447A, [TSK] A0097689Y, [MG] A0153196B  , certify that I have followed the CS 3244 Machine Learning class guidelines for homework assignments.  In particular, I expressly vow that I have followed the Facebook rule in discussing with others in doing the assignment and did not take notes (digital or printed) from the discussions.  

I suggest that I should be graded as follows:

Effort + Leaderboard! 

### References

I have refered to the following list of people and websites in preparing my homework submission:

http://docs.scipy.org/doc/numpy/reference/,
http://stackoverflow.com/,
http://matplotlib.org/api/pyplot_summary.html,
Textbook: Learning From Data,
Lecture's slides
https://keras.io/ - Keras Documentation